In [ ]:
import json

import ipycytoscape
import ipywidgets as W
import networkx as nx
import rdflib
from ipycytoscape import CytoscapeWidget, Edge, Node
from IPython.display import JSON, display
from rdflib.namespace import RDFS

from ipyradiant import (
    FileManager,
    GetOutgoingPredicateObjects,
    InteractiveViewer,
    PathLoader,
)
from ipyradiant.rdf2nx import RDF2NX, URItoID, URItoShortID

## Set up graph using stuff from `RDF_to_NX.ipynb`

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX hum: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (hum:1)  # Luke
            (hum:4)  # Vader
            (film:1) # A New Hope
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))
uri = RDFS.label
ns = {"rdfs": str(RDFS)}
initNs = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "res": "https://swapi.co/resource/",
    "voc": "https://swapi.co/vocabulary/",
    "base": "https://swapi.co/resource/",
}

nx_graph = RDF2NX.convert(simple_graph, namespaces=initNs)

In [ ]:
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)
for node in directed.graph.nodes:
    # deal with inability to handle colons
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(node.data, indent=2)

In [ ]:
iv = InteractiveViewer()
iv.rdf_graph = rdf_graph
iv.cyto_graph = directed

In [ ]:
iv

# Testing

First, manually set the selected node to be Darth Vader and then test the expansions
based on that.

In [ ]:
iv.selected_node = {"data": iv.cyto_graph.graph.nodes[0].data}

Automatically expand on the Darth Vader node.

In [ ]:
iv.expand_button_clicked(None)

Use assertion statements to test that the graph is functioning as it should.

In [ ]:
assert len(iv.cyto_graph.graph.nodes) == 10

Now undo that expansion and test again.

In [ ]:
iv.undo_expansion(None)

In [ ]:
assert len(iv.cyto_graph.graph.nodes) == 3